In [152]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
parser.add_argument('--support', type=float, default=0.2)
parser.add_argument('--confidence',type=float,default=0.2)
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--train_dir TRAIN_DIR] [--support SUPPORT]
                             [--confidence CONFIDENCE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\super\AppData\Roaming\jupyter\runtime\kernel-2e55400c-036a-4e99-97ce-a3ab125e83df.json


SystemExit: 2

E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [139]:
y=np.random.randint(1,6,[1000,])
born=np.random.randint(1960,2003,[1000,])
start_work=born+np.random.randint(18,41,[1000,])
start_work[start_work>2020]=2020
degree=np.random.randint(1,5,[1000,])
dept=np.random.randint(1,6,[1000,])
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])
prof=np.random.random_sample([1000,])

In [140]:
df=pd.DataFrame({
    "born":born,
    "start_work":start_work,
    "dept":dept,
    "degree":degree,
    "prof":prof,
    "mach_cat":mach_cat,
    "meth_cat":meth_cat,
    "time":y
})

In [142]:
#df.to_csv('trainDataHJ.csv',index=False)

In [149]:
def data_pre(path):
    df=pd.read_csv(path)
    now_y=datetime.datetime.now().year
    #transform date to time
    df['born']=df['born'].map(lambda x:now_y-x+1)
    df['start_work']=df['start_work'].map(lambda x:now_y-x+1)
    #iscretization
    bins=[18,25,35,45,60]
    df['born']=pd.cut(df['born'],bins,right=False,labels=['1','2','3','4'])
    bins=[1,3,5,10,41]
    df['start_work']=pd.cut(df['start_work'],bins,right=False,labels=['1','2','3','4'])
    bins=[0,0.6,0.8,0.9,1]
    df['prof']=pd.cut(df['prof'],bins,right=False,labels=['1','2','3','4'])
    #type trans
    df['dept']=df['dept'].apply(str)
    df['mach_cat']=df['mach_cat'].apply(str)
    df['meth_cat']=df['meth_cat'].apply(str)
    df['degree']=df['degree'].apply(str)
    df['time']=df['time'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [159]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [187]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('time') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [193]:
support=0.05
confidence=0.2
train_dir='trainDataHJ.csv'
rules_dir='rulesDataHJ.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

prof_1&meth_cat_2 ==> time_2	59	0.2796208530805687

born_3 ==> time_1	52	0.2708333333333333

meth_cat_1&prof_1 ==> time_4	51	0.2537313432835821

meth_cat_1 ==> time_4	86	0.25219941348973607

meth_cat_1&prof_1 ==> time_5	50	0.24875621890547264

born_4 ==> time_4	87	0.2457627118644068

born_4&start_work_4 ==> time_4	82	0.24404761904761904

prof_1&start_work_4 ==> time_2	74	0.24262295081967214

mach_cat_3 ==> time_4	80	0.24242424242424243

prof_1&born_4&start_work_4 ==> time_4	50	0.23923444976076555

degree_3 ==> time_4	53	0.23873873873873874

prof_1&born_4 ==> time_4	52	0.23853211009174313

meth_cat_3 ==> time_1	75	0.23734177215189872

born_2 ==> time_4	56	0.23628691983122363

degree_4 ==> time_1	64	0.23529411764705882

mach_cat_2 ==> time_4	80	0.23323615160349853

dept_3 ==> time_4	50	0.23148148148148148

dept_3 ==> time_2	50	0.23148148148148148

meth_cat_2 ==> time_2	78	0.22740524781341107

mach_cat_1 ==> time_1	74	0.22629969418960244

degree_2 ==> time_4	56	0